Bloco 1 - Bibliotecas necessarias

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy import stats
from scipy.interpolate import interp1d
from sklearn.metrics import mean_squared_error, r2_score
from scipy.ndimage import uniform_filter1d

Bloco 2 - Funções auxiliares

In [ ]:
# --- 2. Substituição de outliers de Cout pela média local ---
def substituir_outliers_media_local(x, window=3):
    x_corrigido = x.copy()
    q1 = np.percentile(x, 25)
    q3 = np.percentile(x, 75)
    iqr = q3 - q1
    lim_inf = q1 - 1.5 * iqr
    lim_sup = q3 + 1.5 * iqr
    for i in range(len(x)):
        if x[i] < lim_inf or x[i] > lim_sup:
            inicio = max(0, i - window)
            fim = min(len(x), i + window + 1)
            vizinhos = x[inicio:fim]
            vizinhos = vizinhos[(vizinhos >= lim_inf) & (vizinhos <= lim_sup)]
            if len(vizinhos) > 0:
                x_corrigido[i] = np.mean(vizinhos)
    return x_corrigido 

Cout_corrigido = substituir_outliers_media_local(Cout, window=3)

# --- 3. Censura: classificar origem ---
def classificar_origem(Cin, Cout, Cin_lag, Cout_lag, ratio_th=1.2, diff_in=3, diff_ext=2):
    eps = 1e-6
    if (Cin / (Cout + eps)) > ratio_th and (Cin - Cin_lag) > diff_in and (Cout - Cout_lag) < diff_ext or Cin > 45 or Cin > 30.3:
        return "interno"
    elif (Cin / (Cout + eps)) < ratio_th or Cin <= Cout:
        return "externo"
    else:
        return "desconhecido"

Bloco 3 - Loop principal para as 4 residências e 3 frações de partículas

Saídas: 

Resultados Finf (média +/- DP) e intervalos de confiança (95%, 90%, 85% e 68%)
       
Metricas de avaliação do modelo (RMSE, NRMSE e R2)

Gráficos das curvas LOWESS

In [ ]:
# Initialize results list
resultados_lowess = []

# Função LOWESS com Bootstrap
def lowess_with_confidence_bounds(x, y, eval_x, N=300, conf_interval=0.95, lowess_kw=None):
    smoothed = sm.nonparametric.lowess(exog=x, endog=y, xvals=eval_x, **lowess_kw)
    smoothed_values = np.empty((N, len(eval_x)))
    for i in range(N):
        sample = np.random.choice(len(x), len(x), replace=True)
        sampled_x = x[sample]
        sampled_y = y[sample]
        smoothed_values[i] = sm.nonparametric.lowess(
            exog=sampled_x, endog=sampled_y, xvals=eval_x, **lowess_kw
        )
    sorted_values = np.sort(smoothed_values, axis=0)
    bound = int(N * (1 - conf_interval) / 2)
    bottom = sorted_values[bound]
    top = sorted_values[-bound - 1]
    return smoothed, bottom, top

# Parâmetros
arquivos = {
    "R1": "casa1PM.xlsx",
    "R2": "casa2PM.xlsx",
    "R3": "casa3PM.xlsx",
    "R4": "casa4PM.xlsx"
}
fracoes = ["PM1", "PM25", "PM10"]
rotulo_frac = {"PM1": "MP1", "PM25": "MP2.5", "PM10": "MP10"}

# Criar figura de subplots 4x3
fig, axs = plt.subplots(nrows=4, ncols=3, figsize=(20, 16), sharex=False, sharey=False)
plt.subplots_adjust(hspace=0.4, wspace=0.3)

# Loop por casa (linha) e fração (coluna)
for row_idx, (casa_nome, caminho) in enumerate(arquivos.items()):
    df = pd.read_excel(caminho)

    for col_idx, pm in enumerate(fracoes):
        cin_col = f'cin{pm}'
        cout_col = f'cout{pm}'
        ax = axs[row_idx, col_idx]

        # Aqui: ajustar os ticks (tamanho e cor)
        ax.tick_params(axis='both', which='major', labelsize=16, colors='black')
        ax.tick_params(axis='both', which='minor', labelsize=16, colors='black')
        
        if cin_col not in df.columns or cout_col not in df.columns:
            ax.set_visible(False)
            continue

        Cin = df[cin_col].values
        Cout = df[cout_col].values

        # Substituir outliers de Cout por média local
        Cout_corrigido = Cout.copy()
        q1, q3 = np.percentile(Cout, [25, 75])
        iqr = q3 - q1
        lim_inf, lim_sup = q1 - 1.5 * iqr, q3 + 1.5 * iqr
        outliers = (Cout < lim_inf) | (Cout > lim_sup)
        Cout_corrigido[outliers] = uniform_filter1d(Cout, size=3)[outliers]

        # Calcular razão Cin/Cout e aplicar máscara
        raw_ratio = Cin / Cout_corrigido
        mask = (Cout_corrigido > 0) & (~np.isnan(Cin)) & (~np.isnan(Cout_corrigido)) & (raw_ratio <= 1.2) #& (raw_ratio > 0.2)

        x = Cout_corrigido[mask]
        y = raw_ratio[mask]
        
        p90 = np.percentile(x, 90)

        #if len(x) < 20:
            #ax.set_visible(False)
            #continue

        eval_x = np.linspace(x.min(), x.max(), 1000)
        # --- Avaliação do modelo ---
        smoothed, lower, upper = lowess_with_confidence_bounds(x, y, eval_x, N=300, conf_interval=0.95, lowess_kw={'frac': 0.3})
        interp_func = interp1d(eval_x, smoothed, fill_value='extrapolate')
        y_pred = interp_func(x)

        Finf_medio = np.mean(smoothed)
        Finf_std = np.std(smoothed)
        Finf_ci95_min = np.percentile(smoothed, 2.5)
        Finf_ci95_max = np.percentile(smoothed, 97.5)
        Finf_ci90_min = np.percentile(smoothed, 5.0)
        Finf_ci90_max = np.percentile(smoothed, 95.0)
        Finf_ci85_min = np.percentile(smoothed, 7.5)
        Finf_ci85_max = np.percentile(smoothed, 92.5)
        Finf_ci68_min = np.percentile(smoothed, 16.0)
        Finf_ci68_max = np.percentile(smoothed, 84.0)
        rmse = np.sqrt(mean_squared_error(y, y_pred))
        nmrse= rmse/(y.mean() + 1e-6)
        r2 = r2_score(y, y_pred)
        
        # --- Valor-p da regressão simples entre y_pred e y ---
        slope, intercept, r_val, p_val, stderr = stats.linregress(y_pred, y)

        resultados_lowess.append({
            "Casa": casa_nome,
            "PM": pm,
            "Finf_médio": float(Finf_medio),
            "Finf_std": float(Finf_std),
            "Finf_IC95_min": float(Finf_ci95_min),
            "Finf_IC95_max": float(Finf_ci95_max),
            "Finf_IC90_min": float(Finf_ci90_min),
            "Finf_IC90_max": float(Finf_ci90_max),
            "Finf_IC85_min": float(Finf_ci85_min),
            "Finf_IC85_max": float(Finf_ci85_max),
            "Finf_IC68_min": float(Finf_ci68_min),
            "Finf_IC68_max": float(Finf_ci68_max),
            "RMSE": float(rmse),
            "NRMSE": float(nmrse),
            "R²": float(r2),
            "p_valor": float(p_val)
        })

        
        # Plotar curva e intervalo
        ax.scatter(x, y, alpha=0.8, color='black', s=10)
        ax.plot(eval_x, smoothed, color='darkorange', linewidth=2, label='LOWESS')
        ax.fill_between(eval_x, lower, upper, color='gray', alpha=0.5, label='IC 95%')
        ax.axhline(y=p90, color='black', linestyle='--', linewidth=2)
        ax.axvline(x=p90, color='black', linestyle='--', linewidth=2, label='90º percentil de Cout')
        ax.set_ylim(0, 1.5)
        # Títulos e rótulos
        if row_idx == 0:
            ax.set_title(f"{rotulo_frac[pm]}", fontsize=16, color='black')
        if col_idx == 0:
            ax.set_ylabel(f"{casa_nome.upper()}\nCin / Cout", fontsize=16, color='black')
        if row_idx == 3 and col_idx == 1:
            ax.set_xlabel("Cout (µg/m³)", fontsize=16, color='black')
        if row_idx == 0 and col_idx == 2:
            ax.legend(fontsize=16, ncol=2)
# Legenda fora do gráfico
#handles, labels = axs[0, 0].get_legend_handles_labels()
#fig.legend(handles, labels, loc='upper right', ncol=3, fontsize=16)
#fig.suptitle("LOWESS Cin/Cout com IC 95% - Residências (linhas) × Frações (colunas)", fontsize=16, color='black')
plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()

# --- Exibir resultados finais ---
df_resultados = pd.DataFrame(resultados_lowess)
print(df_resultados)

Bloco 4 - Gráficos da Cin e suas parcelas de origem interna e externa

In [ ]:
# Parâmetros
arquivos = {
    "R1": "casa1PM.xlsx",
    "R2": "casa2PM.xlsx",
    "R3": "casa3PM.xlsx",
    "R4": "casa4PM.xlsx"
}
fracoes = ["PM1", "PM25", "PM10"]
rotulo_frac = {"PM1": "MP1", "PM25": "MP2.5", "PM10": "MP10"}

# Criar figura de subplots 4x3
fig, axs = plt.subplots(nrows=4, ncols=3, figsize=(20, 16), sharex=False, sharey=False)
plt.subplots_adjust(hspace=0.4, wspace=0.3)

# Loop por casa (linha) e fração (coluna)
for row_idx, (casa_nome, caminho) in enumerate(arquivos.items()):
    df = pd.read_excel(caminho)

    for col_idx, pm in enumerate(fracoes):
        cin_col = f'cin{pm}'
        cout_col = f'cout{pm}'
        ax = axs[row_idx, col_idx]

        # Aqui: ajustar os ticks (tamanho e cor)
        ax.tick_params(axis='both', which='major', labelsize=16, colors='black')
        ax.tick_params(axis='both', which='minor', labelsize=16, colors='black')

        
        if cin_col not in df.columns or cout_col not in df.columns:
            ax.set_visible(False)
            continue

        Cin = df[cin_col].values
        Cout = df[cout_col].values

        # Substituir outliers de Cout por média local
        Cout_corrigido = Cout.copy()
        q1, q3 = np.percentile(Cout, [25, 75])
        iqr = q3 - q1
        lim_inf, lim_sup = q1 - 1.5 * iqr, q3 + 1.5 * iqr
        outliers = (Cout < lim_inf) | (Cout > lim_sup)
        Cout_corrigido[outliers] = uniform_filter1d(Cout, size=3)[outliers]

        # Calcular razão Cin/Cout e aplicar máscara
        raw_ratio = Cin / Cout_corrigido
        mask = (Cout_corrigido > 0) & (~np.isnan(Cin)) & (~np.isnan(Cout_corrigido)) & (raw_ratio <= 1.2)

        x = df.index[mask]
        y = raw_ratio[mask]
        

        if len(x) < 20:
            ax.set_visible(False)
            continue

        
        eval_x = np.linspace(x.min(), x.max(), 1000)
        smoothed, lower, upper = lowess_with_confidence_bounds(
            x, y, eval_x, N=300, conf_interval=0.95, lowess_kw={"frac": 0.2}
        )
        
        y_pred = Cout_corrigido[mask] * interp1d(eval_x, smoothed, fill_value='extrapolate')(Cout_corrigido[mask])
        Cin_obs = Cin[mask]
        y_pred = Finf_medio*Cout_corrigido[mask]
        #y_pred2 = y_pred.copy()
        y_pred2_std = Finf_std*Cout_corrigido[mask]
        
       # --- Valor-p da regressão simples entre Cin_mod_mean e Cin_ext ---
        slope, intercept, r_val, p_val, stderr = stats.linregress(Cin_obs, y_pred)
        
        if col_idx == 0:
            y_pred2=y_pred-0.1
            Cin_int= Cin_obs - y_pred2
            #Cin_int = np.where(Cin_int < 0, 0, Cin_int)
        elif col_idx == 1:
             y_pred2=y_pred-0.3
             Cin_int= Cin_obs - y_pred2
             #Cin_int = np.where(Cin_int < 0, 0, Cin_int)
        else:
             y_pred2=y_pred-4
             Cin_int= Cin_obs - y_pred2
             #Cin_int = np.where(Cin_int < 0, 0, Cin_int)
        
   
        # Plotar curva e intervalo
        ax.plot(x, Cin_obs, alpha=0.8, color='black', label='Cin Observado', linewidth=1)
        ax.plot(x, y_pred2, color='darkorange', linewidth=2, label='Cin_ext LOWESS')
        ax.fill_between(x,
                        y_pred2 - y_pred2_std,
                        y_pred2 + y_pred2_std,
                        color='darkorange', alpha=0.3, label='±1σ')
        ax.plot(x, Cin_int, color='orangered', linewidth=2, label='Cin_int LOWESS', linestyle='--')
        #ax.fill_between(x, Cin_int - y_pred2_std, Cin_int + y_pred2_std, color='blue', alpha=0.3, label='IC 95%')
        
        for spine in ['bottom', 'left', 'right', 'top']:
            ax.spines[spine].set_color('black')
            
        ax.axhline(0, color='black', linestyle=':', linewidth=2)
        
        # Títulos e rótulos
        if row_idx == 0:
            ax.set_title(f"{rotulo_frac[pm]}", fontsize=16, color='black')
        if col_idx == 0:
            ax.set_ylabel(f"{casa_nome.upper()}\nConcentração (µg/m³)", fontsize=16, color='black')
        if row_idx == 3:
            ax.set_xlabel("Tempo (índice)", fontsize=16, color='black')
        
        if row_idx == 0 and col_idx == 2:
            ax.legend(fontsize=16)

# Legenda fora do gráfico
#handles, labels = axs[0, 0].get_legend_handles_labels()
#fig.legend(handles, labels, loc='upper right', ncol=3, fontsize=16)
#fig.suptitle("LOWESS Cin com IC 95%\nResidências (linhas) × Frações (colunas)", fontsize=16, color='black')
plt.tight_layout(rect=[0.05, 0.05, 1, 0.94])
plt.show()
